In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from tensorflow.keras.datasets import mnist
train, test = mnist.load_data()

In [4]:
x_train = train[0].reshape(60000, 28, 28, 1) / 255 # 正規化
x_val = test[0].reshape(10000, 28, 28, 1) / 255 # 正規化

In [5]:
t_train = train[1]
t_val = test[1]

In [6]:
x_train[0].shape

(28, 28, 1)

In [14]:
type(x_train), type(x_val), type(t_train), type(t_val)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [7]:
import os
import random

def reset_seed(seed=0):

    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed) #　random関数のシードを固定
    np.random.seed(seed) #numpyのシードを固定
    tf.random.set_seed(seed) #tensorflowのシードを固定

In [8]:
from tensorflow.keras import models, layers

# シードの固定
reset_seed(0)
tf.keras.backend.clear_session()

# モデルの構築
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(64, (2, 2), activation='relu')(inputs)
x = layers.MaxPool2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(100, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [11]:
optimizer = keras.optimizers.Adam(lr=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [12]:
history = model.fit(x_train, t_train, batch_size=256, epochs=5, validation_data=(x_val, t_val))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 184us/sample - loss: 0.1927 - accuracy: 0.9410 - val_loss: 0.0758 - val_accuracy: 0.9761
Epoch 2/5
60000/60000 [==============================] - 10s 170us/sample - loss: 0.0567 - accuracy: 0.9823 - val_loss: 0.0815 - val_accuracy: 0.9741
Epoch 3/5
60000/60000 [==============================] - 10s 170us/sample - loss: 0.0386 - accuracy: 0.9876 - val_loss: 0.0699 - val_accuracy: 0.9787
Epoch 4/5
60000/60000 [==============================] - 10s 168us/sample - loss: 0.0227 - accuracy: 0.9924 - val_loss: 0.0753 - val_accuracy: 0.9799
Epoch 5/5
60000/60000 [==============================] - 10s 170us/sample - loss: 0.0229 - accuracy: 0.9923 - val_loss: 0.0788 - val_accuracy: 0.9802


ResNetを真似してモデル構築：失敗してます

In [15]:
# モデルの構築

# ブロック 1
inputs = keras.Input(shape=(28, 28, 1), name='img')
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.Conv2D(64, 3, activation='relu')(x)
block_1_output = layers.MaxPooling2D(3)(x)

# ブロック 2
x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
block_2_output = layers.add([x, block_1_output]) # 1 つ目のブロックの出力値を加算

# ブロック 3
x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_2_output)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
block_3_output = layers.add([x, block_2_output]) # 2 つ目のブロックの出力値を加算

# 最終ブロック
x = layers.Conv2D(64, 3, activation='relu')(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs, name='resnet')

In [18]:
optimizer = keras.optimizers.Adam(lr=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [19]:
history = model.fit(x_train, t_train, batch_size=256, epochs=5, verbose=1, validation_data=(x_val, t_val))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 53s 889us/sample - loss: 2.3303 - accuracy: 0.1111 - val_loss: 2.3012 - val_accuracy: 0.1135
Epoch 2/5
60000/60000 [==============================] - 52s 868us/sample - loss: 2.3017 - accuracy: 0.1119 - val_loss: 2.3015 - val_accuracy: 0.1135
Epoch 3/5
60000/60000 [==============================] - 52s 868us/sample - loss: 2.3017 - accuracy: 0.1124 - val_loss: 2.3014 - val_accuracy: 0.1135
Epoch 4/5
60000/60000 [==============================] - 52s 871us/sample - loss: 2.3017 - accuracy: 0.1124 - val_loss: 2.3015 - val_accuracy: 0.1135
Epoch 5/5
60000/60000 [==============================] - 52s 868us/sample - loss: 2.3017 - accuracy: 0.1117 - val_loss: 2.3011 - val_accuracy: 0.1135
